# HNSCC Progressors and NonProgressors

## Annotate HNSCC TCGA Tumors as Progressors or NonProgressors

### McWeeney Lab, Oregon Health & Science University

** Author: Gabrielle Choonoo (choonoo@ohsu.edu) **

## Introduction

This is the step-by-step workflow for annotating HNSCC tumors as Progressors or NonProgressors based on clinical data from TCGA. 

Required Files:
* Clinical Data (.txt): [[Can download from TCGA using this method]](https://github.com/gchoonoo/HNSCC_Clinical_Data_Notebook)
* This notebook (Annotate_Progressors_Notebook.ipynb): [[Download here]](https://raw.githubusercontent.com/gchoonoo/HNSCC_Annotate_Tumor_Progression/master/Annotate_Progressors_Notebook.ipynb)

**Note: this notebook can also be downloaded as an R script (only the code blocks seen below will be included):[[Download R script here]](https://raw.githubusercontent.com/gchoonoo/HNSCC_Annotate_Tumor_Progression/master/annotate_progressors.r)

** All code is available on GitHub: [https://github.com/gchoonoo/HNSCC_Annotate_Tumor_Progression](https://github.com/gchoonoo/HNSCC_Annotate_Tumor_Progression) **

## Annotate HNSCC Tumor Progressors and NonProgressors

### Original criteria: Patients were first classified as progressor or nonprogressor based on follow-up annotation, specifically the presence or absence of a new tumor event. We required annotation to confirm the tumor event (days to new tumor and/or new tumor anatomical location). All patients were required to have treatment annotation in addition to the follow-up data. https://www.ncbi.nlm.nih.gov/pubmed/26747525

# Read in cleaned clinical data

In [ ]:
read.delim(file="raw_clinical_data.txt", sep="\t", header=T, stringsAsFactors = F) -> hnsc_data

# Add new column for annotation

In [ ]:
hnsc_data$Progression_FINAL_new = NA

### Criteria:

### Progressor if they had any one of these:
* "days_to_new_tumor_event_after_initial_treatment" not NA
* "new_tumor_event_after_initial_treatment" = YES
* "new_neoplasm_event_occurrence_anatomic_site" not NA
* "followup_treatment_success" = Progressive Disease or Persistent Disease

### Nonprogressor if they had any one of these and weren't annotated as Progressor based on above criteria:
* "days_to_new_tumor_event_after_initial_treatment" = NA
* "new_tumor_event_after_initial_treatment"  = NO
* "new_neoplasm_event_occurrence_anatomic_site" = NA
* "followup_treatment_success"  = Complete Remission/Response, Stable Disease, Partial Remission/Response, or NA

# Annotate Progressors

In [ ]:
hnsc_data[which(!is.na(hnsc_data[, "days_to_new_tumor_event_after_initial_treatment"]) | hnsc_data[,"new_tumor_event_after_initial_treatment"] == "YES" | !is.na(hnsc_data[,"new_neoplasm_event_occurrence_anatomic_site"]) | hnsc_data[,"followup_treatment_success"] == "Progressive Disease" | hnsc_data[,"followup_treatment_success"] == "Persistent Disease"),"Progression_FINAL_new"] <- "Progressor"

# Annotate NonProgressors

In [ ]:
hnsc_data[which((is.na(hnsc_data[, "days_to_new_tumor_event_after_initial_treatment"]) | hnsc_data[,"new_tumor_event_after_initial_treatment"] == "NO" | is.na(hnsc_data[,"new_neoplasm_event_occurrence_anatomic_site"]) | hnsc_data[,"followup_treatment_success"] == "Complete Remission/Response" | hnsc_data[,"followup_treatment_success"] == "Stable Disease" | hnsc_data[,"followup_treatment_success"] == "Partial Remission/Response" | is.na(hnsc_data[,"followup_treatment_success"])) & is.na(hnsc_data[,"Progression_FINAL_new"])),"Progression_FINAL_new"] <- "NonProgressor"

# Save data

In [ ]:
write.table(file="clinical_data_annotated.txt", x=hnsc_data, sep="\t", quote=F, row.names=F)